In [2]:
# Clone the GitHub repository
!git clone https://github.com/AVI68/radar_nowcasting.git

# Navigate to the repository directory
import os
os.chdir("radar_nowcasting")

# Verify the repository content
!ls


Cloning into 'radar_nowcasting'...
Updating files:  30% (60/198)
Updating files:  31% (62/198)
Updating files:  32% (64/198)
Updating files:  33% (66/198)
Updating files:  34% (68/198)
Updating files:  35% (70/198)
Updating files:  36% (72/198)
Updating files:  37% (74/198)
Updating files:  38% (76/198)
Updating files:  39% (78/198)
Updating files:  40% (80/198)
Updating files:  41% (82/198)
Updating files:  42% (84/198)
Updating files:  43% (86/198)
Updating files:  44% (88/198)
Updating files:  45% (90/198)
Updating files:  46% (92/198)
Updating files:  47% (94/198)
Updating files:  48% (96/198)
Updating files:  49% (98/198)
Updating files:  50% (99/198)
Updating files:  51% (101/198)
Updating files:  52% (103/198)
Updating files:  53% (105/198)
Updating files:  54% (107/198)
Updating files:  55% (109/198)
Updating files:  56% (111/198)
Updating files:  57% (113/198)
Updating files:  58% (115/198)
Updating files:  59% (117/198)
Updating files:  60% (119/198)
Updating files:  61% (121

Code
Data
README.md


In [3]:
# Install required libraries
!pip install pysteps pyproj contextily matplotlib

In [5]:
# Import Necessary Libraries
import os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from pysteps.visualization import plot_precip_field, quiver
from pprint import pprint
from contextily import add_basemap
import pyproj
from matplotlib.ticker import MaxNLocator, FuncFormatter

# Import utility functions from the repository
import utility

ImportError: cannot import name 'utility' from 'code' (C:\Users\Utente\anaconda3\envs\avi_ppt_dwnsc\Lib\code.py)

In [6]:
# Define the root directory for the repository data
root_path = os.path.join(os.getcwd(), "Data")

# Define metadata files
metadata_X = utility.get_matadata(os.path.join(root_path, "radarmappatipo.tif"), type="X")
metadata_C = utility.get_matadata(os.path.join(root_path, "civilpromap.tiff"), type="C")


NameError: name 'utility' is not defined

In [ ]:
# Choose the event date
date = datetime.strptime("20191024_0415", "%Y%m%d_%H%M")

# Define parameters for X-band radar data
data_source_X = "UNICA_SG"
fn_pattern_X = "%Y%m%d_%H%M"
f_ext_X = ".png"

# Load X-band radar data
R_X, metadata_X = utility.import_files_by_date(date, root_path, data_source_X, fn_pattern_X, f_ext_X, metadata_X, timestep=5, num_prev_files=9)

# Noise removal using Watershed technique
R_X_clean = np.empty_like(R_X)
for t in range(R_X.shape[0]):
    R_X_clean[t, :, :] = utility.noise_remove(R_X[t, :, :], type="Watershed")

# Convert Digital Number to Reflectivity (dBZ)
R_X_dbz = utility.dn_to_dbz(R_X_clean)

# Convert to rain rate
R_X_R, metadata_X = pysteps.utils.conversion.to_rainrate(R_X_dbz, metadata_X)

# Log-transform the data
image_R, metadata_X = pysteps.utils.transformation.dB_transform(R_X_R, metadata_X, threshold=0.01, zerovalue=-15.0)

# Print metadata
pprint(metadata_X)


In [ ]:
# Estimate motion field using Lucas-Kanade optical flow
oflow_method = pysteps.motion.get_method("LK")
V1 = oflow_method(image_R[-3:, :, :])

# Plot precipitation field and motion field
fig, ax = plt.subplots(figsize=(10, 8))

# Precipitation field
plot_precip_field(
    R_X_dbz[-1, :, :], 
    ptype="intensity", 
    geodata=metadata_X, 
    units="dBZ", 
    title="Lucas Kanade Optical Flow Method", 
    ax=ax, 
    colorscale="pysteps"
)

# Motion field (quiver plot)
quiver(V1, geodata=metadata_X, step=45, ax=ax)

# Convert meter-based ticks to degrees
def convert_meters_to_degrees(x, y, crs_proj, crs_geo="EPSG:4326"):
    transformer = pyproj.Transformer.from_crs(crs_proj, crs_geo, always_xy=True)
    return transformer.transform(x, y)

def format_ticks(value, tick_number, axis):
    lon, lat = convert_meters_to_degrees(value, 0, metadata_X["projection"]) if axis == "x" else convert_meters_to_degrees(0, value, metadata_X["projection"])
    return f"{lon:.2f}" if axis == "x" else f"{lat:.2f}"

# Customize ticks
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: format_ticks(x, 0, "x")))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: format_ticks(y, 0, "y")))
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))

# Add basemap
add_basemap(ax, crs=metadata_X["projection"], source=add_basemap.providers.OpenStreetMap.Mapnik)

# Show plot
plt.show()